# How users feel? A sentiment Analysis on Twitter v3
## Giacomo Manfucci
### A challenge from Open Data Playgroud

In [48]:
#
#    Salve a tutto il team di Open Data Playgroud,
#    Sono Giacomo Manfucci, uno studente di statistica presso l'Università degli Studi di Firenze.
#    Questo è il codice python del mio tentativo 
#

## Importo il train dataset e opero una breve esplorazione dei dati

### Due caratteristiche

     1   tweet:
     
         La componente testuale del tweet su cui fare analisi del sentiment.
         
     2   label:
     
         L'etichetta che classifica il tweet:
             0 -> negative
             1 -> neutral
             2 -> positive
            
In realtà mi accorgerò dopo che le etichette sono 4 e quindi lavorerò su queste 4 etichette che vanno da 0 a 3.

In [49]:
import pandas as pd

#Importo il file e lo salvo come un df pandas
df_train = pd.read_csv(r"C:\Users\giaco\OneDrive\Desktop\Sfide_ML\dataset_OpenDataPlay\train_complete.csv")
df_train.head()

,tweet,label
0,“Worry is a down payment on a problem you may ...,2
1,My roommate: it's okay that we can't spell bec...,0
2,No but that's so cute. Atsu was probably shy a...,1
3,Rooneys fucking untouchable isn't he? Been fuc...,0
4,it's pretty depressing when u hit pan on ur fa...,3


In [50]:
#Vediamo quante righe e colonne il df ha
df_train.shape

(3631, 2)

In [51]:
#Costruisco la tabella di frequenze delle etichette
freq = df_train['label'].value_counts() 
print(freq) 

0    1560
3     944
1     805
2     322
Name: label, dtype: int64


## Voglio ora ripulire il testo dei tweet

Voglio eliminare qualsiasi carattere non testo, tranne per le faccine che decido di mettere alla fine del testo. Uso quindi delle espressioni regolari che mi permettano di fare questo. Anche se non è una scelta elegante (elimino anche la punteggiatura e l'ordine), opto per la scelta più immediata e vedo come funziona.
        

In [52]:
#Creo così la mia funzione di gestione delle espressioni regolari

#Questa funzione mi eliminerà eventuali rimasugli di HTML (non ci dovrebbero essere),
#poi mi trasferisce le emoticon alla fine del tweet e infine
#mi porta tutto il testo in minuscolo e rimuove qualche altro particolare
#

import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

In [53]:
#Trasformo così i miei dati testuali tramite la funzione appena costruita
df_train['tweet'] = df_train['tweet'].apply(preprocessor)

df_train.head()

,tweet,label
0,worry is a down payment on a problem you may ...,2
1,my roommate it s okay that we can t spell beca...,0
2,no but that s so cute atsu was probably shy ab...,1
3,rooneys fucking untouchable isn t he been fuck...,0
4,it s pretty depressing when u hit pan on ur fa...,3


# Addestramento, ottimizzazione e stima delle prestazioni

## Addestramento

Addestrerò il mio modello usando il metodo bag-of-words con rappresentazione 1-gram (rendendo quindi giustificata la perdita di ordine che verrà effettuata), inoltre userò la valutazione della rilevanza delle parole tramite la tecnica Term Frequency - Inverse Document Frequency (TF-IDF). 

L'algoritmo di classificazione sarà quello a regressione logistica per problemi multi-classe (OvA, One vs All).

## Ottimizzazione 

Ottimizzerò il mio modello tramite la ricerca a griglia.


In [54]:
#Definisco la tokenizzazione classica e quella di stemming di Porter per separare le parole.

from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.split()


def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]


In [55]:
#Divido il mio dataset in un dataset di addestramento e uno di test 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_train['tweet'], df_train['label'],         
                                                test_size=0.3,          
                                                random_state=0, 
                                                stratify=df_train['label'])             

In [56]:
# Addestro e ottimizzo gli iperparametri del mio modello di classificazione, di analisi del sentiment

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

#Questa è la lista di dizionari degli iperparametri che mi servirà per la ricerca a griglia

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

#Costruisco quindi la mia breve Pipeline

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0, solver='liblinear', multi_class='ovr'))])

#Faccio la ricerca a griglia con una convalida incrociata con k = 10, valore abbastanza standard, forse potevo usare un k<10, 
#ma va bene così.

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=10,
                           verbose=2,
                           n_jobs=-1) #Ho impostato n_jobs = -1 per chi ha la fortuna di poter lavorare in parallelo

gs_lr_tfidf.fit(X_train, y_train)

Fitting 10 folds for each of 24 candidates, totalling 240 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(multi_class='ovr',
                                                           random_state=0,
                                                           solver='liblinear'))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [None],
                          'vect__tokenizer': [<function tokenizer at 0x000001DC7189DB80>,
                                              <function tokenizer_porter at 0x000001DC7189D9D0>]},
                         {'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          

In [57]:
#Stampo a schermo i migliori parametri e le relative performance 
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

#Quindi costruisco il mio modello con i valori degli iperparametri migliori e misuro la performance.
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Best parameter set: {'clf__C': 1.0, 'clf__penalty': 'l1', 'vect__ngram_range': (1, 1), 'vect__norm': None, 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer_porter at 0x000001DC7189D9D0>, 'vect__use_idf': False} 
CV Accuracy: 0.704
Test Accuracy: 0.697


## Riaddestro ora sull'intero dataset 
Come buona norma. Non ricordo se l'implementazione precedente lo fa di default (non credo), comunque nel dubbio lo faccio manualmente, tanto il costo computazionale non è alto.

In [58]:
clf.fit(df_train['tweet'], df_train['label'])

Pipeline(steps=[('vect',
                 TfidfVectorizer(lowercase=False, norm=None,
                                 tokenizer=<function tokenizer_porter at 0x000001DC7189D9D0>,
                                 use_idf=False)),
                ('clf',
                 LogisticRegression(multi_class='ovr', penalty='l1',
                                    random_state=0, solver='liblinear'))])

In [59]:
#Questa operazione è inutile ai fini del controllo della performance, ma era per controllare che tutto fosse in ordine
clf.score(X_test, y_test)

0.8779816513761468

# Importo ora il dataset sul quale farò le mie predizioni del sentiment


In [60]:
with open(r'C:\Users\giaco\OneDrive\Desktop\Sfide_ML\dataset_OpenDataPlay\test_text.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

X_sub = pd.DataFrame({'tweet': lines})

X_sub.head()


,tweet
0,#Deppression is real. Partners w/ #depressed p...
1,@user Interesting choice of words... Are you c...
2,My visit to hospital for care triggered #traum...
3,@user Welcome to #MPSVT! We are delighted to h...
4,What makes you feel #joyful? \n


In [61]:
X_sub.shape

(1421, 1)

### Dopo aver letto il file e portato in un df pandas ora faccio le predizioni

In [64]:
y_sub = clf.predict(X_sub['tweet'])
y_sub = pd.DataFrame({'label': y_sub})

y_sub.head()

,label
0,0
1,0
2,3
3,1
4,3


### Unisco in un unico df X_sub e y_sub e esporto il df in un file csv come richiesto

In [65]:
df_sub = pd.concat([X_sub, y_sub], axis=1)

df_sub.head()

,tweet,label
0,#Deppression is real. Partners w/ #depressed p...,0
1,@user Interesting choice of words... Are you c...,0
2,My visit to hospital for care triggered #traum...,3
3,@user Welcome to #MPSVT! We are delighted to h...,1
4,What makes you feel #joyful? \n,3


In [69]:
df_sub.to_csv(r"C:\Users\giaco\OneDrive\Desktop\Sfide_ML\submits_OpenDataPlay\submit_1_ODP.csv", index=False)